In [ ]:
import arcpy
from sys import argv

plots = ["p1","p2","p3"]
dates = ["july20","sept20"]

fname=["p1_july20","p1_sept20","p2_july20","p2_sept20","p3_july20","p3_sept20"]

wksp=r"D:\ETREC"

cellsize=0.01

coordsys="PROJCS['New Projected Coordinate System',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Local'],PARAMETER['False_Easting',0.0],PARAMETER['False_Northing',0.0],PARAMETER['Scale_Factor',1.0],PARAMETER['Azimuth',45.0],PARAMETER['Longitude_Of_Center',-75.0],PARAMETER['Latitude_Of_Center',40.0],UNIT['Meter',1.0]]"


arcpy.env.overwriteOutput = True
arcpy.env.cellSize = cellsize
arcpy.env.extent = "MAXOF"
arcpy.env.workspace = wksp
arcpy.env.outputCoordinateSystem=coordsys
arcpy.scratchWorkspace=fr"{wksp}\scratch\scratch.gdb"

In [ ]:
# CREATE INITIAL SCRATCH DEM AND DELINEATE PLOT BASINS

for i in fname:
    inlas=fr"{wksp}\LAS\{i}.las"

    Update_Input_Dataset_or_Feature_Class = arcpy.management.DefineProjection(in_dataset=inlas,
        coor_system=coordsys)

    arcpy.management.LasDatasetStatistics(in_las_dataset=inlas,
        calculation_type="OVERWRITE_EXISTING_STATS", 
        out_file=fr"{wksp}\scratch\{i}stats.txt", 
        summary_level="DATASET", 
        delimiter="COMMA", 
        decimal_separator="DECIMAL_POINT")

    demfull=fr"{wksp}\DEM_unfilt\dem_{i}_avg_1cm.tif"
    
    arcpy.conversion.LasDatasetToRaster(in_las_dataset=inlas, 
        out_raster=demfull, 
        value_field="ELEVATION", 
        interpolation_type="BINNING AVERAGE LINEAR", 
        data_type="FLOAT", 
        sampling_type="CELLSIZE", 
        sampling_value=cellsize, 
        z_factor=1)

    print(fr"{i} DEM complete")

    Fill_dem = "memory\\Fill_dem"
    Fill = Fill_dem
    with arcpy.EnvManager(extent=Extent):
        Fill_dem = arcpy.sa.Fill(in_surface_raster=demfull, z_limit=None)
        Fill_dem.save(Fill)
    
       # Process: Flow Direction (Flow Direction) (sa)
    fdr = "memory\\fdr"
    Flow_Direction = fdr
    Output_drop_raster = ""
    with arcpy.EnvManager(extent=Extent):
        fdr = arcpy.sa.FlowDirection(in_surface_raster=Fill_dem, 
            force_flow="NORMAL", 
            out_drop_raster=Output_drop_raster, 
            flow_direction_type="D8")
        fdr.save(Flow_Direction) 

        # Process: Basin (Basin) (sa)
    bas_p_plot_date_tif = "memory\\bas"
    Basin = bas_p_plot_date_tif
    with arcpy.EnvManager(extent=Extent):
        bas_p_plot_date_tif = arcpy.sa.Basin(in_flow_direction_raster=fdr)
        bas_p_plot_date_tif.save(Basin)

        # Process: Raster to Polygon (Raster to Polygon) (conversion)
    basp_p_plot_date_shp = fr"{wksp}\scratch\scratch.gdb\basp_{i}"
    with arcpy.EnvManager(extent=Extent, 
        outputMFlag="Disabled", 
        outputZFlag="Disabled"):
        arcpy.conversion.RasterToPolygon(in_raster=bas_p_plot_date_tif,
            out_polygon_features=basp_p_plot_date_shp, 
            simplify="NO_SIMPLIFY", 
            raster_field="VALUE", 
            create_multipart_features="SINGLE_OUTER_PART", 
            max_vertices_per_feature=None)

    print(fr"{i} basins complete")

print("all done")

In [ ]:
# CREATE PLOT BOUNDARIES FROM BASINS

for p in plots:
        # Process: Union (Union) (analysis)
    ufeat = [fr"{wksp}\scratch\scratch.gdb\basp_{p}_{dates[0]}", fr"{wksp}\scratch\scratch.gdb\basp_{p}_{dates[1]"]
    basp_union = fr"{wksp}\scratch\scratch.gdb\{p}_union"
    arcpy.analysis.Union(in_features=ufeat, 
        out_feature_class=basp_union, 
        join_attributes="ALL", 
        cluster_tolerance="", 
        gaps="GAPS")

    pbound = fr"{wksp}\bounds\{p}_bounds.shp" 
    arcpy.analysis.Select(in_features=basp_union, 
        out_feature_class=pbound, 
        where_clause="Shape_Area > 100")
    
    print(f"{p} boundary done.")


In [ ]:
# START WHITEBOX TOOLS

import os
import sys
from WBT.whitebox_tools import WhiteboxTools

wbt = WhiteboxTools()

In [ ]:
# CLIP LAS TO BOUNDARY POLYGONS AND WBT REMOVE OUTLIERS FILTER

for p in plots:
    for d in dates:
        n = p +"_"+ d
        
        inlas = fr"{wksp}\LAS\{n}.las"
        remout = fr"{wksp}\LAS_filt\{n}_ro.las"
        lasclip = fr"{wksp}\LAS\{n}_clipw.las"
        bounds= fr"{wksp}\bounds\{p}_bounds.shp"

        wbt.clip_lidar_to_polygon(
            i=inlas, 
            polygons=bounds, 
            output=lasclip)

        wbt.lidar_point_stats(
            i=lasclip, 
            resolution=0.01, 
            num_points=False, 
            num_pulses=False, 
            avg_points_per_pulse=False, 
            z_range=True, 
            intensity_range=False, 
            predom_class=False)

        wbt.lidar_remove_outliers(
            i=lasclip, 
            output=remout, 
            radius=0.03, 
            elev_diff=0.03, 
            use_median=True, 
            classify=True)

       print(f"{n} filter complete.")
    
    print(f"all of {p} filtered.") 

In [ ]:
#   CREATE DEMs

for p in plots:
    for d in dates:
        n = p +"_"+ d

        arcpy.env.extent = fr"{wksp}\bounds\{p}_bounds.shp"

        lasd=fr"{wksp}\LAS_filt\{n}_ro.las"

        arcpy.management.DefineProjection(in_dataset=lasd,
            coor_system=coordsys)

        arcpy.management.LasDatasetStatistics(in_las_dataset=lasd,
            calculation_type="OVERWRITE_EXISTING_STATS", 
            #out_file=fr"{wksp}\scratch\{n}_stats_fin.txt", 
            summary_level="DATASET", 
            delimiter="COMMA", 
            decimal_separator="DECIMAL_POINT")

        tempdem="memory\\demfilt"

        arcpy.conversion.LasDatasetToRaster(in_las_dataset=lasd, 
            out_raster="memory\\demfilt", 
            value_field="ELEVATION", 
            interpolation_type="BINNING AVERAGE LINEAR", 
            data_type="FLOAT", 
            sampling_type="CELLSIZE", 
            sampling_value=cellsize, 
            z_factor=1)

# Extract by Mask  

        extractdem=fr"{wksp}\DEM_final\dem_{n}_nosmooth.tif"
        mask=fr"{wksp}\bounds\{p}_bounds.shp"
        outExtractByMask = arcpy.sa.ExtractByMask(tempdem, mask)
        outExtractByMask.save(extractdem)
       
        arcpy.BuildPyramidsandStatistics_management(extractdem, resample_technique="BILINEAR")

        print(f"{n} dem ready.")
    
    print(f"{p} surfaces created.") 

In [ ]:
# FEATURE PRESERVING SMOOTHING

for n in fname:

    wbt.feature_preserving_smoothing(
        dem=fr"{wksp}\DEM_final\dem_{n}_nosmooth.tif", 
        output=fr"{wksp}\DEM_final\dem_{n}_smooth.tif", 
        filter=11, 
        norm_diff=15.0, 
        num_iter=3, 
        max_diff=0.5, 
        zfactor=None, 
    )

In [ ]:
plots = ["p1","p2","p3"]
dates = ["july20","sept20"]

for p in plots:
    print(f"testing {p} and {dates[0]} and {dates[1]}")
    
    input1 = fr"{wksp}/DEM_final/dem_{p}_{dates[0]}_nosmooth.tif"
    input2 = fr"{wksp}/DEM_final/dem_{p}_{dates[1]}_nosmooth.tif"
    output = fr"{wksp}/scratch/TTEST_{p}_{dates[0]}_{dates[1]}"

        # Whitebox Tools T-Test
        # Returns an HTML file to the output folder 
    wbt.paired_sample_t_test(
    input1, 
    input2, 
    output, 
    num_samples=500)


In [ ]:
for p in plots:
    print(f"subtracting {p} {dates[0]} and {dates[1]}")

    inrast1 = fr"{wksp}\DEM_final\dem_{p}_{dates[0]}_nosmooth.tif"
    inrast2 = fr"{wksp}\DEM_final\dem_{p}_{dates[1]}_nosmooth.tif"
    output = fr"{wksp}\results\demdiff_{p}_{dates[1]}_{dates[0]}_nosmooth.tif"
    arcpy.env.extent = fr"{wksp}\bounds\{p}_bounds.shp"

    outMinus = arcpy.sa.Minus(inrast2, inrast1)
    outMinus.save(output)        

    arcpy.BuildPyramidsandStatistics_management(output,
        resample_technique="BILINEAR")  
 

